In [91]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import xlrd 
import json


# # Import API key
# from api_keys import api_key


In [93]:
#import data from url to get the state abbr
url="http://worldpopulationreview.com/static/states/name-abbr.json"
d = requests.get(url).json()

states_abbr = pd.DataFrame(d.items() , columns=["state","abbr"])
states_abbr.head()

,state,abbr
0,Alabama,AL
1,Alaska,AK
2,American Samoa,AS
3,Arizona,AZ
4,Arkansas,AR


In [94]:
#import excel file with obesity, unemployment, education
xls = pd.ExcelFile('Resources/2018 County Health Rankings Data - v2.xls')
df = pd.read_excel(xls, 'Ranked Measure Data', skiprows=1)


In [95]:
df.head()


,FIPS,State,County,Years of Potential Life Lost Rate,95% CI - Low,95% CI - High,Quartile,Years of Potential Life Lost Rate (Black),Years of Potential Life Lost Rate (Hispanic),Years of Potential Life Lost Rate (White),...,95% CI - High.20,Quartile.33,% Drive Alone (Black),% Drive Alone (Hispanic),% Drive Alone (White),# Workers who Drive Alone,% Long Commute - Drives Alone,95% CI - Low.21,95% CI - High.21,Quartile.34
0,1001.0,Alabama,Autauga,9409.294766,8492.111635,10326.477898,2,11806.223285,NaN,9148.381743,...,88.431482,4,82.494851,NaN,82.539683,21082.0,40.8,36.698437,44.901563,3
1,1003.0,Alabama,Baldwin,7467.596664,7025.900408,7909.292920,1,10335.702691,2827.348216,7484.117065,...,86.017804,2,85.567010,68.864800,84.646255,73058.0,40.1,37.614952,42.585048,3
2,1005.0,Alabama,Barbour,8929.474982,7633.929691,10225.020273,1,9141.333085,NaN,8896.000532,...,85.796041,1,80.341227,NaN,85.656228,7271.0,35.1,29.740946,40.459054,2
3,1007.0,Alabama,Bibb,11741.938889,10057.801626,13426.076153,4,11297.166352,NaN,11923.067966,...,89.905820,2,NaN,NaN,NaN,7044.0,48.8,40.894758,56.705242,4
4,1009.0,Alabama,Blount,9359.122343,8463.490290,10254.754396,1,NaN,5133.144570,9600.725685,...,87.622499,3,NaN,79.539642,81.580997,18384.0,60.2,55.818947,64.581053,4


In [96]:
#rename columns used for analysis
df.rename(columns={ df.columns[114]: "Unemployment Percent" }, inplace = True)
df.rename(columns={ df.columns[34]: "Obesity Percent" }, inplace = True)
df.rename(columns={ df.columns[108]: "Some College Percent" }, inplace = True)
df.rename(columns={ df.columns[104]: "High School Grad Percent" }, inplace = True)

Unemployment_Percent=df["Unemployment Percent"]
Obesity_Percent=df["Obesity Percent"]
Some_College=df["Some College Percent"]
High_School=df["High School Grad Percent"]
State_Name=df["State"]
County=df["County"]
state_abbr=states_abbr["abbr"]

#create new Dataframe with just information needed
stats_df= pd.DataFrame({"State": State_Name,
                        "State Abbr": state_abbr,
                        "County": County,
                        "Unemployment (%)": Unemployment_Percent,
                        "Obesity (%)": Obesity_Percent,
                        "Some College (%)": Some_College,
                        "High School Graduate (%)": High_School
                       })

stats_df.head()

,State,State Abbr,County,Unemployment (%),Obesity (%),Some College (%),High School Graduate (%)
0,Alabama,AL,Autauga,5.271161,36.4,61.942019,90.000000
1,Alabama,AK,Baldwin,5.415263,29.3,63.757100,85.000000
2,Alabama,AS,Barbour,8.648518,44.2,39.723661,86.173469
3,Alabama,AZ,Bibb,6.637117,38.4,49.821371,85.000000
4,Alabama,AR,Blount,5.520897,35.8,53.866457,91.206250


In [142]:
SNAP = pd.ExcelFile('Resources/cntysnap.xls')
SNAP_df = pd.read_excel(SNAP, skiprows=2)
SNAP_df = SNAP_df.join(SNAP_df["Name"].str.split(",", 1, expand=True).rename(columns={0:'County', 1:'State Abbr'}))
SNAP_df["County"] = SNAP_df["County"].str.replace("County", "")
SNAP_df.head()


,State FIPS code,County FIPS code,Name,July 2016,July 2015,July 2014,July 2013,July 2012,July 2011,July 2010,...,July 2001,July 2000,July 1999,July 1998,July 1997,July 1995,July 1993,July 1989,County,State Abbr
0,1,0,Alabama,813769.0,864350.0,893684.0,905604.0,914740.0,905658.0,859695.0,...,435235.0,405325.0,396931.0,409160.0,434632.0,510271.0,561128.0,435296.0,Alabama,None
1,1,1,"Autauga County, AL",7844.0,8015.0,8145.0,8055.0,8079.0,8060.0,7593.0,...,3035.0,2814.0,2700.0,2589.0,2771.0,3611.0,4608.0,3739.0,Autauga,AL
2,1,3,"Baldwin County, AL",21406.0,22920.0,24563.0,24402.0,23169.0,22184.0,20604.0,...,4129.0,4178.0,4372.0,4850.0,5201.0,6928.0,8053.0,5548.0,Baldwin,AL
3,1,5,"Barbour County, AL",6675.0,6730.0,6725.0,6740.0,6843.0,6465.0,6324.0,...,4637.0,4415.0,4490.0,4788.0,4947.0,5268.0,5499.0,5056.0,Barbour,AL
4,1,7,"Bibb County, AL",3740.0,3794.0,4085.0,4175.0,4269.0,4336.0,4152.0,...,1765.0,1685.0,1631.0,1776.0,1999.0,2085.0,2503.0,2056.0,Bibb,AL


In [143]:
reduced_SNAP_df = SNAP_df.loc[:, ["State Abbr", "County", "July 2016"]]
reduced_SNAP_df = reduced_SNAP_df[reduced_SNAP_df["State Abbr"].astype(str) != "None"]
reduced_SNAP_df.head()


,State Abbr,County,July 2016
1,AL,Autauga,7844.0
2,AL,Baldwin,21406.0
3,AL,Barbour,6675.0
4,AL,Bibb,3740.0
5,AL,Blount,7037.0
